과정 : 데이터를 한 1/2만 남겨두고 없앤다. 근데 없애도 되나? 없애면 분석 측면에서 곤란하긴 함.

In [1]:
import pandas as pd

In [2]:
principal_data = pd.read_csv("./dataset/title.principals.tsv", sep = "\t")  ## heavy file

In [3]:
name_data = pd.read_csv("./dataset/name.basics.tsv", sep = "\t")
title_data = pd.read_csv("./dataset/title.basics.tsv", sep = "\t")
crews_data = pd.read_csv("./dataset/title.crew.tsv", sep = "\t")
rating_data = pd.read_csv("./dataset/title.ratings.tsv", sep = "\t")
episode_data = pd.read_csv("./dataset/title.episode.tsv", sep = "\t")

/tmp/ipykernel_92009/2426090303.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  title_data = pd.read_csv("./dataset/title.basics.tsv", sep = "\t")


기대되는 개체 : 영화, 감독, 배우, 에피소드(?)

기대되는 관계 : 제작(감독-영화), 출현(배우-영화)

`principal_data`는 영화의 제목과 기여자에 대한 정보를 담고 있으며, 기본키는 {tconst, ordering}이다.

In [19]:
principal_data

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0005690,producer,producer,\N
3,tt0000001,4,nm0374658,cinematographer,director of photography,\N
4,tt0000002,1,nm0721526,director,\N,\N
...,...,...,...,...,...,...
85947715,tt9916880,18,nm0996406,director,principal director,\N
85947716,tt9916880,19,nm1482639,writer,\N,\N
85947717,tt9916880,20,nm2586970,writer,books,\N
85947718,tt9916880,21,nm1594058,producer,producer,\N


속성의 원자성 만족(?)

In [23]:
len(set(principal_data.characters))

4066577

In [5]:
len(set(principal_data.job))

42680

In [6]:
principal_data.shape

(85947720, 6)

`name_data`는 스태프들의 이름이다. 주 포지션으로 구분되며 참여작품도 같이 기재되어 있다. 뒤의 두 속성은 속성의 원자성을 충족하지 못하므로 재가공이 요구된다.

In [7]:
name_data

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0083922,tt0050976,tt0069467"
...,...,...,...,...,...,...
13515558,nm9993714,Romeo del Rosario,\N,\N,"animation_department,art_department","tt11657662,tt14069590,tt2455546"
13515559,nm9993716,Essias Loberg,\N,\N,\N,\N
13515560,nm9993717,Harikrishnan Rajan,\N,\N,cinematographer,tt8736744
13515561,nm9993718,Aayush Nair,\N,\N,cinematographer,tt8736744


In [16]:
name_data.primaryProfession.str.split(',')

0                 [actor, miscellaneous, producer]
1           [actress, soundtrack, archive_footage]
2            [actress, music_department, producer]
3                [actor, writer, music_department]
4                        [writer, director, actor]
                             ...                  
13515558    [animation_department, art_department]
13515559                                      [\N]
13515560                         [cinematographer]
13515561                         [cinematographer]
13515562                                  [writer]
Name: primaryProfession, Length: 13515563, dtype: object

`title_data`는 영화의 제목에 대한 테이블임. 기본키는 `tconst`

In [8]:
title_data.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


> 장르가 원소성 불충족

In [9]:
len(set(title_data.tconst)) == title_data.shape[0]  ## 모든 열의 값이 다름. 기본키로 사용 가능함(최소 차원 & 튜플의 유일성)

True

> 해당 테이블은 "영화" 개체로 잡아도 문제가 없을 것 같음.

In [10]:
title_data.loc[title_data.tconst == "tt0041951"]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
41214,tt0041951,tvEpisode,The Tenderfeet,The Tenderfeet,0,1949,\N,30,Western


`crews_data`는 영화의 감독과 작가를 나타낸 데이터이다.

In [11]:
crews_data  ## 작가와 감독 데이터이다. 두 릴레이션을 합친 개형과 비슷

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N
...,...,...,...
10150897,tt9916848,nm1485677,"nm9187127,nm1485677,nm9826385,nm9299459,nm1628284"
10150898,tt9916850,nm1485677,"nm9187127,nm1485677,nm9826385,nm1628284"
10150899,tt9916852,nm1485677,"nm9187127,nm1485677,nm9826385,nm9299459,nm1628284"
10150900,tt9916856,nm10538645,nm6951431


In [27]:
crews_data_directors = crews_data.loc[:, ["tconst", 'directors']]
crews_data_writers = crews_data.loc[:, ["tconst", 'writers']]

In [37]:
featured_crew_directors = crews_data_directors.set_index("tconst").apply(lambda x : x.str.split(',').explode()).reset_index()
featured_crew_writers = crews_data_writers.set_index("tconst").apply(lambda x : x.str.split(',').explode()).reset_index()

In [38]:
featured_crew_writers

,tconst,writers
0,tt0000001,\N
1,tt0000002,\N
2,tt0000003,\N
3,tt0000004,\N
4,tt0000005,\N
...,...,...
17761362,tt9916852,nm9299459
17761363,tt9916852,nm1628284
17761364,tt9916856,nm6951431
17761365,tt9916880,nm1482639


> 두번째와 세번째 열이 속성의 원자성을 충족하지 못함

`rating_data`는 영화의 평점에 대한 데이터이다.

In [12]:
rating_data

,tconst,averageRating,numVotes
0,tt0000001,5.7,2058
1,tt0000002,5.6,277
2,tt0000003,6.5,2015
3,tt0000004,5.4,180
4,tt0000005,6.2,2785
...,...,...,...
1439917,tt9916730,7.0,12
1439918,tt9916766,7.1,23
1439919,tt9916778,7.2,36
1439920,tt9916840,7.0,9


> 평가한 사람이 없어 추가적인 개체를 불러오기엔 부적합하다.

`episode_data`는 영화의 에피소드 수를 기재한 자료이다.

In [13]:
episode_data.head()

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0041951,tt0041038,1,9
1,tt0042816,tt0989125,1,17
2,tt0042889,tt0989125,\N,\N
3,tt0043426,tt0040051,3,42
4,tt0043631,tt0989125,2,16


> 이건 그냥 영화 개체에 추가적인 정보가 될 것 같다.

영화 - 감독 : 촬영

영화 - 배우 : 출연